class for defining the GridWorld (4x4) Environment and its variables

In [1]:
import numpy as np

class GridWorld:
    def __init__(self, size=4):
        self.size = size
        self.states = [(i, j) for i in range(size) for j in range(size)]
        self.actions = ['U', 'D', 'L', 'R']
        self.terminal_states = [(0, 0), (size - 1, size - 1)]
        self.gamma = 1.0

    def is_terminal(self, state):
        return state in self.terminal_states

    def next_state(self, state, action):
        if self.is_terminal(state):
            return state, 0
        i, j = state
        if action == 'U':
            i = max(i - 1, 0)
        elif action == 'D':
            i = min(i + 1, self.size - 1)
        elif action == 'L':
            j = max(j - 1, 0)
        elif action == 'R':
            j = min(j + 1, self.size - 1)
        new_state = (i, j)
        reward = 0 if new_state in self.terminal_states else -1
        return new_state, reward


Iterative Policy Evaluation to calculate the number of iterations needed to converge the values

In [2]:
def initialize_random_policy(env):
    policy = {}
    for state in env.states:
        if env.is_terminal(state):
            policy[state] = {}
        else:
            policy[state] = {a: 0.25 for a in env.actions}
    return policy

def policy_evaluation(env, policy, theta=1e-4):
    V = np.zeros((env.size, env.size))
    iteration = 0
    while True:
        delta = 0
        for state in env.states:
            if env.is_terminal(state):
                continue
            v = 0
            for action, action_prob in policy[state].items():
                next_state, reward = env.next_state(state, action)
                v += action_prob * (reward + env.gamma * V[next_state])
            delta = max(delta, np.abs(v - V[state]))
            V[state] = v
        iteration += 1
        if delta < theta:
            break
    return V, iteration

# Example Usage
env = GridWorld()
policy = initialize_random_policy(env)
V, iterations = policy_evaluation(env, policy)
print("Value Function after Policy Evaluation:\n", V)
print("Iterations to Converge:", iterations)


Value Function after Policy Evaluation:
 [[  0.         -12.99934883 -18.99906386 -20.9989696 ]
 [-12.99934883 -16.99920093 -18.99913239 -18.99914232]
 [-18.99906386 -18.99913239 -16.9992679  -12.9994534 ]
 [-20.9989696  -18.99914232 -12.9994534    0.        ]]
Iterations to Converge: 114


Policy Iteration using Policy Evaluation

In [3]:
def policy_iteration(env):
    policy = initialize_random_policy(env)
    while True:
        V, _ = policy_evaluation(env, policy)
        policy_stable = True
        for state in env.states:
            if env.is_terminal(state):
                continue
            old_action = max(policy[state], key=policy[state].get)
            action_values = {}
            for action in env.actions:
                next_state, reward = env.next_state(state, action)
                action_values[action] = reward + env.gamma * V[next_state]
            best_action = max(action_values, key=action_values.get)
            new_policy = {a: (1.0 if a == best_action else 0.0) for a in env.actions}
            if policy[state] != new_policy:
                policy_stable = False
            policy[state] = new_policy
        if policy_stable:
            break
    return policy, V

# Example Usage
optimal_policy, optimal_V = policy_iteration(env)
print("Optimal Policy from Policy Iteration:")
for state in sorted(optimal_policy.keys()):
    print(f"{state}: {optimal_policy[state]}")
print("\nOptimal Value Function:\n", optimal_V)


Optimal Policy from Policy Iteration:
(0, 0): {}
(0, 1): {'U': 0.0, 'D': 0.0, 'L': 1.0, 'R': 0.0}
(0, 2): {'U': 0.0, 'D': 0.0, 'L': 1.0, 'R': 0.0}
(0, 3): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(1, 0): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(1, 1): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(1, 2): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(1, 3): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(2, 0): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(2, 1): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(2, 2): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(2, 3): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(3, 0): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(3, 1): {'U': 0.0, 'D': 0.0, 'L': 0.0, 'R': 1.0}
(3, 2): {'U': 0.0, 'D': 0.0, 'L': 0.0, 'R': 1.0}
(3, 3): {}

Optimal Value Function:
 [[ 0.  0. -1. -2.]
 [ 0. -1. -2. -1.]
 [-1. -2. -1.  0.]
 [-2. -1.  0.  0.]]


Value Iteration

In [4]:
def value_iteration(env, theta=1e-4):
    V = np.zeros((env.size, env.size))
    while True:
        delta = 0
        for state in env.states:
            if env.is_terminal(state):
                continue
            action_values = []
            for action in env.actions:
                next_state, reward = env.next_state(state, action)
                action_values.append(reward + env.gamma * V[next_state])
            max_value = max(action_values)
            delta = max(delta, np.abs(max_value - V[state]))
            V[state] = max_value
        if delta < theta:
            break

    policy = {}
    for state in env.states:
        if env.is_terminal(state):
            policy[state] = {}
            continue
        action_values = {}
        for action in env.actions:
            next_state, reward = env.next_state(state, action)
            action_values[action] = reward + env.gamma * V[next_state]
        best_action = max(action_values, key=action_values.get)
        policy[state] = {a: (1.0 if a == best_action else 0.0) for a in env.actions}
    return policy, V

# Example Usage
value_policy, value_V = value_iteration(env)
print("Optimal Policy from Value Iteration:")
for state in sorted(value_policy.keys()):
    print(f"{state}: {value_policy[state]}")
print("\nOptimal Value Function:\n", value_V)


Optimal Policy from Value Iteration:
(0, 0): {}
(0, 1): {'U': 0.0, 'D': 0.0, 'L': 1.0, 'R': 0.0}
(0, 2): {'U': 0.0, 'D': 0.0, 'L': 1.0, 'R': 0.0}
(0, 3): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(1, 0): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(1, 1): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(1, 2): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(1, 3): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(2, 0): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(2, 1): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(2, 2): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(2, 3): {'U': 0.0, 'D': 1.0, 'L': 0.0, 'R': 0.0}
(3, 0): {'U': 1.0, 'D': 0.0, 'L': 0.0, 'R': 0.0}
(3, 1): {'U': 0.0, 'D': 0.0, 'L': 0.0, 'R': 1.0}
(3, 2): {'U': 0.0, 'D': 0.0, 'L': 0.0, 'R': 1.0}
(3, 3): {}

Optimal Value Function:
 [[ 0.  0. -1. -2.]
 [ 0. -1. -2. -1.]
 [-1. -2. -1.  0.]
 [-2. -1.  0.  0.]]


#Conclusion

In this experiment, we successfully implemented and analyzed two dynamic programming techniques — Policy Iteration and Value Iteration — to solve the classic GridWorld problem in Reinforcement Learning. We began by defining the environment as a 4x4 grid, modeling the transitions, rewards, and terminal states. Using an initial random policy, we evaluated the value function iteratively until convergence and recorded the number of iterations required. This helped us understand the process of policy evaluation and how value functions evolve with repeated updates. Further, we implemented Policy Iteration by alternating between policy evaluation and policy improvement steps, ultimately leading to an optimal policy that maximizes cumulative rewards. We then applied Value Iteration, which combines evaluation and improvement in a single update step, and observed that it yielded similar optimal values and policies more efficiently. Through this lab, we gained hands-on experience with Markov Decision Processes (MDPs), the Bellman equations, and the foundational ideas of dynamic programming in RL, which are crucial for building more advanced agents in real-world environments.